this is data science capstone project
Project predicts that considering all the conditions like weather,road quality etc, what are the chances of getting in to an accident and what kind of damage can be done on you by the accident using data of past years

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

In [66]:
path = 'https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv' #importing the data
df = pd.read_csv(path)


In [45]:
df.shape #to check the total no of rows and columns in data

(194673, 38)

In [46]:
df.dtypes #to check the data type of each attribute so as to get more understanding

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [67]:
df.drop(['SEVERITYCODE.1','SEVERITYDESC','COLLISIONTYPE'] , axis=1, inplace=True) #dropping these 3 columns as information is already conveyed by SEVERITYCODE 

In [68]:
df.drop(['OBJECTID','INCKEY','COLDETKEY','REPORTNO','STATUS','INTKEY','EXCEPTRSNCODE','EXCEPTRSNDESC','SDOT_COLCODE','SDOT_COLDESC','SDOTCOLNUM','ST_COLCODE','ST_COLDESC','SEGLANEKEY','CROSSWALKKEY'] , axis=1,inplace=True)
#dropping these columns as most are keys and have less or no significance on the data

In [49]:
df.shape #as we can see the data has decresed to columns

(194673, 20)

In [69]:
df.drop(['ADDRTYPE','LOCATION','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','PEDROWNOTGRNT','HITPARKEDCAR'] , axis=1,inplace=True) #dropping columns thgat will not help in prediction

In [71]:
df.rename(columns = {"X":"LONGITUDE","Y":"LATITUDE"}, inplace=True)        #making data more expressible

In [76]:
df.tail(20) #final dataset to be used 

,SEVERITYCODE,LONGITUDE,LATITUDE,INCDATE,INCDTTM,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
194653,2,-122.322097,47.649615,2018/12/10 00:00:00+00,12/10/2018 12:47:00 AM,Mid-Block (not related to intersection),NaN,Y,Overcast,Wet,Dark - Street Lights On,NaN
194654,1,-122.312679,47.719414,2019/01/10 00:00:00+00,1/10/2019,At Intersection (intersection related),NaN,NaN,Clear,Dry,Daylight,NaN
194655,2,-122.380016,47.664879,2019/01/20 00:00:00+00,1/20/2019,At Intersection (intersection related),NaN,NaN,Clear,Dry,Daylight,NaN
194656,2,-122.340474,47.614496,2019/01/23 00:00:00+00,1/23/2019 10:15:00 AM,At Intersection (intersection related),NaN,N,Clear,Dry,Daylight,NaN
194657,1,-122.337137,47.610709,2018/11/26 00:00:00+00,11/26/2018 3:37:00 AM,Mid-Block (not related to intersection),NaN,N,Raining,Wet,Dark - Street Lights On,NaN
194658,1,-122.302554,47.584099,2019/01/18 00:00:00+00,1/18/2019 9:57:00 PM,Mid-Block (not related to intersection),NaN,N,Raining,Wet,Dark - Street Lights On,NaN
194659,1,-122.298955,47.717456,2018/11/27 00:00:00+00,11/27/2018 5:30:00 PM,Mid-Block (not related to intersection),NaN,N,Raining,Wet,Dark - Street Lights On,NaN
194660,2,-122.330518,47.619543,2018/12/23 00:00:00+00,12/23/2018 12:52:00 PM,Mid-Block (not related to intersection),NaN,N,Raining,Wet,Other,NaN
194661,2,-122.306092,47.617881,2019/03/22 00:00:00+00,3/22/2019 5:19:00 PM,Mid-Block (but intersection related),NaN,Y,Clear,Dry,Daylight,NaN
194662,1,-122.337880,47.625793,2018/12/05 00:00:00+00,12/5/2018 5:09:00 PM,Mid-Block (but intersection related),NaN,N,Clear,Dry,Dusk,NaN


#now data will be prepared


In [73]:
#now longitude and latitude have 5334 missing values so replacing them by mean
avg_long = df["LONGITUDE"].astype("float").mean(axis = 0)


# replace NaN by mean value in "stroke" column
df["LONGITUDE"].replace(np.nan, avg_long, inplace = True)

In [74]:
avg_lat = df["LATITUDE"].astype("float").mean(axis = 0)


# replace NaN by mean value in "stroke" column
df["LATITUDE"].replace(np.nan, avg_lat, inplace = True)

In [62]:
#now unknown values in categorical variables are replaced by mode
for column in ['JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [79]:
#replacing nan values with 'N' in INATTENTIONIND, UNDERINFL,SPEEDING
df['INATTENTIONIND'] = df['INATTENTIONIND'].fillna('N')
df['UNDERINFL'] = df['UNDERINFL'].fillna('N')
df['SPEEDING'] = df['SPEEDING'].fillna('N')

In [80]:
df.tail(20)

,SEVERITYCODE,LONGITUDE,LATITUDE,INCDATE,INCDTTM,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
194653,2,-122.322097,47.649615,2018/12/10 00:00:00+00,12/10/2018 12:47:00 AM,Mid-Block (not related to intersection),N,Y,Overcast,Wet,Dark - Street Lights On,N
194654,1,-122.312679,47.719414,2019/01/10 00:00:00+00,1/10/2019,At Intersection (intersection related),N,N,Clear,Dry,Daylight,N
194655,2,-122.380016,47.664879,2019/01/20 00:00:00+00,1/20/2019,At Intersection (intersection related),N,N,Clear,Dry,Daylight,N
194656,2,-122.340474,47.614496,2019/01/23 00:00:00+00,1/23/2019 10:15:00 AM,At Intersection (intersection related),N,N,Clear,Dry,Daylight,N
194657,1,-122.337137,47.610709,2018/11/26 00:00:00+00,11/26/2018 3:37:00 AM,Mid-Block (not related to intersection),N,N,Raining,Wet,Dark - Street Lights On,N
194658,1,-122.302554,47.584099,2019/01/18 00:00:00+00,1/18/2019 9:57:00 PM,Mid-Block (not related to intersection),N,N,Raining,Wet,Dark - Street Lights On,N
194659,1,-122.298955,47.717456,2018/11/27 00:00:00+00,11/27/2018 5:30:00 PM,Mid-Block (not related to intersection),N,N,Raining,Wet,Dark - Street Lights On,N
194660,2,-122.330518,47.619543,2018/12/23 00:00:00+00,12/23/2018 12:52:00 PM,Mid-Block (not related to intersection),N,N,Raining,Wet,Other,N
194661,2,-122.306092,47.617881,2019/03/22 00:00:00+00,3/22/2019 5:19:00 PM,Mid-Block (but intersection related),N,Y,Clear,Dry,Daylight,N
194662,1,-122.337880,47.625793,2018/12/05 00:00:00+00,12/5/2018 5:09:00 PM,Mid-Block (but intersection related),N,N,Clear,Dry,Dusk,N


In [83]:
#we see that date and location have no effect in prediction so we will drop them in data frame
test = ['SEVERITYCODE', 'SPEEDING','ROADCOND','LIGHTCOND','WEATHER','INATTENTIONIND','UNDERINFL']
df_data = df[test]

In [84]:
df_data.head()

,SEVERITYCODE,SPEEDING,ROADCOND,LIGHTCOND,WEATHER,INATTENTIONIND,UNDERINFL
0,2,N,Wet,Daylight,Overcast,N,N
1,1,N,Wet,Dark - Street Lights On,Raining,N,0
2,1,N,Dry,Daylight,Overcast,N,0
3,1,N,Dry,Daylight,Clear,N,N
4,2,N,Wet,Daylight,Raining,N,0


In [92]:
# We assign new values to roadcond
df_data['ROADCOND'].replace(to_replace=['Wet','Dry','Unknown','Snow/Slush','Ice','Other','Sand/Mud/Dirt','Standing Water','Oil'], value = ['Dangerous','Normal','Normal','Dangerous','Dangerous','Normal','Dangerous','Dangerous','Dangerous'], inplace=True)

In [100]:
df_data['ROADCOND'].value_counts()

Normal       144732
Dangerous     49941
Name: ROADCOND, dtype: int64

In [94]:
df_data.head()

,SEVERITYCODE,SPEEDING,ROADCOND,LIGHTCOND,WEATHER,INATTENTIONIND,UNDERINFL
0,2,N,Dangerous,Daylight,Overcast,N,N
1,1,N,Dangerous,Dark - Street Lights On,Raining,N,0
2,1,N,Normal,Daylight,Overcast,N,0
3,1,N,Normal,Daylight,Clear,N,N
4,2,N,Dangerous,Daylight,Raining,N,0


In [95]:
df_data['LIGHTCOND'].value_counts()

Daylight                    121307
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [98]:
# We assign new values to roadcond
df_data['LIGHTCOND'].replace(to_replace=['Daylight','Dark - Street Lights On','Unknown','Dusk','Dawn','Dark - No Street Lights','Dark - Street Lights Off','Other','Dark - Unknown Lighting'], value = ['Day','Night','Night','Night','Day','Night','Night','Night','Night'], inplace=True)

In [99]:
df_data['LIGHTCOND'].value_counts()

Day      123809
Night     70864
Name: LIGHTCOND, dtype: int64

In [101]:
df_data['WEATHER'].value_counts()

Clear                       116216
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [102]:
# We assign new values to roadcond
df_data['WEATHER'].replace(to_replace=['Clear','Raining','Overcast','Unknown','Snowing','Other','Fog/Somg/Smoke','Sleet/Hail/Freezing Rain','Blowing Sand/Dirt','Severe Crosswind','Partly Cloudy'], value = ['Good','Bad','Good','Bad','Bad','Bad','Bad','Bad','Bad','Bad','good'], inplace=True)

In [103]:
df_data['WEATHER'].value_counts()

Good              143930
Bad                50169
Fog/Smog/Smoke       569
good                   5
Name: WEATHER, dtype: int64

In [104]:
# We assign new values to roadcond
df_data['WEATHER'].replace(to_replace=['Fog/Smog/Smoke','good'], value = ['Bad','Good'], inplace=True)

In [111]:
df_data['SPEEDING'].value_counts()

N    185340
Y      9333
Name: SPEEDING, dtype: int64

In [108]:
# We assign new values to roadcond
df_data['UNDERINFL'].replace(to_replace=['N','Y'], value = ['0','1'], inplace=True)

In [109]:
df_data['UNDERINFL'].value_counts()

0    185552
1      9121
Name: UNDERINFL, dtype: int64

In [110]:
df_data.head(10)

,SEVERITYCODE,SPEEDING,ROADCOND,LIGHTCOND,WEATHER,INATTENTIONIND,UNDERINFL
0,2,N,Dangerous,Day,Good,N,0
1,1,N,Dangerous,Night,Bad,N,0
2,1,N,Normal,Day,Good,N,0
3,1,N,Normal,Day,Good,N,0
4,2,N,Dangerous,Day,Bad,N,0
5,1,N,Normal,Day,Good,N,0
6,1,N,Dangerous,Day,Bad,N,0
7,2,N,Normal,Day,Good,N,0
8,1,N,Normal,Day,Good,N,0
9,2,N,Normal,Day,Good,N,0


In [119]:

df_data['ROADCOND'].replace(to_replace=['Dangerous','Normal'],value=[0,1],inplace=True)
df_data['LIGHTCOND'].replace(to_replace=['Day','Night'],value=[1,0],inplace=True)
df_data['WEATHER'].replace(to_replace=['Good','Bad'],value=[1,0],inplace=True)
df_data['INATTENTIONIND'].replace(to_replace=['N','Y'],value=[0,1],inplace=True)


TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [126]:
df_data.tail(10)

,SEVERITYCODE,SPEEDING,ROADCOND,LIGHTCOND,WEATHER,INATTENTIONIND,UNDERINFL
194663,2,N,0,1,0,1,0
194664,1,N,1,1,1,0,0
194665,1,N,1,1,1,0,0
194666,2,N,0,1,1,0,0
194667,1,N,0,0,0,0,1
194668,2,N,1,1,1,0,0
194669,1,N,0,1,0,1,0
194670,2,N,1,1,1,0,0
194671,2,N,1,0,1,0,0
194672,1,N,0,1,1,0,0


In [128]:
df_data["UNDERINFL"] = df_data["UNDERINFL"].astype("int")

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [129]:
df_data.dtypes

SEVERITYCODE       int64
SPEEDING          object
ROADCOND           int64
LIGHTCOND          int64
WEATHER            int64
INATTENTIONIND     int64
UNDERINFL          int64
dtype: object

In [132]:
df_data['SPEEDING'].replace(to_replace=['N','Y'],value=[0,1],inplace=True)

In [133]:
df_data.dtypes

SEVERITYCODE      int64
SPEEDING          int64
ROADCOND          int64
LIGHTCOND         int64
WEATHER           int64
INATTENTIONIND    int64
UNDERINFL         int64
dtype: object

In [138]:
df_data.head(20)

,SEVERITYCODE,SPEEDING,ROADCOND,LIGHTCOND,WEATHER,INATTENTIONIND,UNDERINFL
0,2,0,0,1,1,0,0
1,1,0,0,0,0,0,0
2,1,0,1,1,1,0,0
3,1,0,1,1,1,0,0
4,2,0,0,1,0,0,0
5,1,0,1,1,1,0,0
6,1,0,0,1,0,0,0
7,2,0,1,1,1,0,0
8,1,0,1,1,1,0,0
9,2,0,1,1,1,0,0


In [140]:
df_data['SPEEDING'].value_counts()

0    185340
1      9333
Name: SPEEDING, dtype: int64

In [146]:
x = df_data[['SPEEDING','ROADCOND','LIGHTCOND','WEATHER','INATTENTIONIND','UNDERINFL']]
y = df_data['SEVERITYCODE'].values.astype(str)
x = preprocessing.StandardScaler().fit(x).transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)


Train set: (155738, 6) (155738,)
Test set: (38935, 6) (38935,)


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


In [147]:
#tree model
Tree_model = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
Tree_model.fit(x_train, y_train)
predicted = Tree_model.predict(x_test)
Tree_f1 = f1_score(y_test, predicted, average='weighted')
Tree_acc = accuracy_score(y_test, predicted)

In [148]:
#Logistic Regression
LR_model = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
predicted = LR_model.predict(x_test)
LR_f1 = f1_score(y_test, predicted, average='weighted')
LR_acc = accuracy_score(y_test, predicted)

In [150]:
#KNN methodology
KNN_model = KNeighborsClassifier(n_neighbors = 4).fit(x_train, y_train)
predicted = KNN_model.predict(x_test)
KNN_f1 = f1_score(y_test, predicted, average='weighted')
KNN_acc = accuracy_score(y_test, predicted)

In [151]:
results = {
    "Method of Analisys": ["KNN", "Decision Tree", "LogisticRegression"],
    "F1-score": [KNN_f1, Tree_f1, LR_f1],
    "Accuracy": [KNN_acc, Tree_acc, LR_acc]
}

results = pd.DataFrame(results)
results

,Method of Analisys,F1-score,Accuracy
0,KNN,0.615139,0.675947
1,Decision Tree,0.577235,0.700064
2,LogisticRegression,0.579591,0.698241


In [152]:
results = {
    "Intercept": LR_model.intercept_,
    "SPEEDING ": LR_model.coef_[:,0],
    "ROADCOND ": LR_model.coef_[:,1],
    "LIGHTCOND ": LR_model.coef_[:,2],
    "WEATHER ": LR_model.coef_[:,3],
    "INATTENTIONIND": LR_model.coef_[:,4],
    "UNDERINFL": LR_model.coef_[:,5],
}

results = pd.DataFrame(results)
results

,Intercept,SPEEDING,ROADCOND,LIGHTCOND,WEATHER,INATTENTIONIND,UNDERINFL
0,-0.874066,0.067521,-0.221704,0.170609,0.235691,0.080582,0.113277
